# Aula 7 - DPR

[Unicamp - IA368DD: Deep Learning aplicado a sistemas de busca.](https://www.cpg.feec.unicamp.br/cpg/lista/caderno_horario_show.php?id=1779)

Autor: Marcus Vinícius Borela de Castro

[Repositório no github](https://github.com/marcusborela/deep_learning_em_buscas_unicamp)


## Enunciado do Exercício

Fazer o finetuning de um buscador denso

Usar como treino o dataset "tiny" do MS MARCO
https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv

Avaliar o modelo no TREC-COVID, e comparar os resultados com o BM25 e doc2query

Comparar busca "exaustiva" (semelhança do vetor query com todos os vetores do corpus) com a busca aproximada (Approximate Nearest Neighbor - ANN)

Para a busca aproximada, usar os algoritmos existentes na biblioteca sentence-transformers (ex: hnswlib) OU implemente um você mesmo (Bonus!)

Dicas:

    Usar a média dos vetores da última camada (conhecido como mean pooling) do transformer para representar queries e passagens; Alternativamente, usar apenas o vetor do [CLS] da última cada.
    Tente inicialmente uma loss fácil de implementar, como a entropia-cruzada
    Começar o treino a partir do microsoft/MiniLM-L12-H384-uncased
    Avaliar o pipeline usando um modelo já bem treinado: sentence-transformers/all-mpnet-base-v2
    Comparar resultados usando semelhança de cosseno e produto escalar como funções de similaridade
    Para checar se seu codigo de avaliação está correto, comparar o seu desempenho com o do modelo já treinado no MS MARCO:   https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2; O nDCG@10 no TREC-COVID deve ser ~0.47
    Usar a biblioteca do sentence-transformers para avaliar o modelo


## Fase

Fine tunning no modelo

# Organizando o ambiente

## Importação dos pacotes

In [1]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
# !pip install transformers

In [2]:
import math
import random
import torch
import numpy as np
from torch.utils.data import DataLoader


In [3]:
import itertools

In [4]:
from transformers import AutoTokenizer,  AutoModel

/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import os

In [6]:
import transformers

In [7]:
from tqdm import tqdm

In [8]:
import time

In [9]:
DIRETORIO_TRABALHO = '/home/borela/fontes/deep_learning_em_buscas_unicamp/local/dpr'


In [10]:
if os.path.exists(DIRETORIO_TRABALHO):
    print('pasta já existia!')
else:
    os.makedirs(DIRETORIO_TRABALHO)
    print('pasta criada!')


pasta já existia!


In [11]:
from psutil import virtual_memory


In [12]:
def mostra_memoria(lista_mem=['cpu']):
  """
  Esta função exibe informações de memória da CPU e/ou GPU, conforme parâmetros fornecidos.

  Parâmetros:
  -----------
  lista_mem : list, opcional
      Lista com strings 'cpu' e/ou 'gpu'. 
      'cpu' - exibe informações de memória da CPU.
      'gpu' - exibe informações de memória da GPU (se disponível).
      O valor padrão é ['cpu'].

  Saída:
  -------
  A função não retorna nada, apenas exibe as informações na tela.

  Exemplo de uso:
  ---------------
  Para exibir informações de memória da CPU:
      mostra_memoria(['cpu'])

  Para exibir informações de memória da CPU e GPU:
      mostra_memoria(['cpu', 'gpu'])
  
  Autor: Marcus Vinícius Borela de Castro

  """  
  if 'cpu' in lista_mem:
    vm = virtual_memory()
    ram={}
    ram['total']=round(vm.total / 1e9,2)
    ram['available']=round(virtual_memory().available / 1e9,2)
    # ram['percent']=round(virtual_memory().percent / 1e9,2)
    ram['used']=round(virtual_memory().used / 1e9,2)
    ram['free']=round(virtual_memory().free / 1e9,2)
    ram['active']=round(virtual_memory().active / 1e9,2)
    ram['inactive']=round(virtual_memory().inactive / 1e9,2)
    ram['buffers']=round(virtual_memory().buffers / 1e9,2)
    ram['cached']=round(virtual_memory().cached/1e9 ,2)
    print(f"Your runtime RAM in gb: \n total {ram['total']}\n available {ram['available']}\n used {ram['used']}\n free {ram['free']}\n cached {ram['cached']}\n buffers {ram['buffers']}")
    print('/nGPU')
    gpu_info = !nvidia-smi
  if 'gpu' in lista_mem:
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Not connected to a GPU')
    else:
      print(gpu_info)


In [13]:
mostra_memoria(['cpu', 'gpu'])

Your runtime RAM in gb: 
 total 67.35
 available 57.57
 used 8.67
 free 12.58
 cached 45.6
 buffers 0.5
/nGPU
Tue Apr 18 18:18:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.39.01    Driver Version: 510.39.01    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:02:00.0 Off |                  N/A |
| 58%   50C    P8    37W / 370W |     60MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+--

In [14]:
current_dir = os.getcwd()
print("Current directory:", current_dir)

Current directory: /home/borela/fontes/deep_learning_em_buscas_unicamp/code/aula7_dpr


## Fixando as seeds

In [15]:
import random
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader

In [16]:
def inicializa_seed(num_semente:int=123):
  """
  Inicializa as sementes para garantir a reprodutibilidade dos resultados do modelo.
  Essa é uma prática recomendada, já que a geração de números aleatórios pode influenciar os resultados do modelo.
  Além disso, a função também configura as sementes da GPU para garantir a reprodutibilidade quando se utiliza aceleração por GPU. 
  
  Args:
      num_semente (int): número da semente a ser utilizada para inicializar as sementes das bibliotecas.
  
  References:
      http://nlp.seas.harvard.edu/2018/04/03/attention.html
      https://github.com/CyberZHG/torch-multi-head-attention/blob/master/torch_multi_head_attention/multi_head_attention.py#L15
  """
  # Define as sementes das bibliotecas random, numpy e pytorch
  random.seed(num_semente)
  np.random.seed(num_semente)
  torch.manual_seed(num_semente)
  
  # Define as sementes da GPU
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

  #torch.cuda.manual_seed(num_semente)
  #Cuda algorithms
  #torch.backends.cudnn.deterministic = True


In [17]:
num_semente=123
inicializa_seed(num_semente)

## Definindo Hiperparâmetros iniciais

In [18]:
def inicia_hparam()->dict:
  # Inicialização dos parâmetros
  hparam = {}
  hparam["num_workers_dataloader"] = 0
  hparam["device"] = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  if torch.cuda.is_available(): print(torch. cuda. get_device_name(hparam["device"]))    
  return hparam

In [19]:
hparam=inicia_hparam()

NVIDIA GeForce RTX 3090


## Preparando para debug e display

In [20]:
import pandas as pd

https://zohaib.me/debugging-in-google-collab-notebook/

In [21]:
# !pip install -Uqq ipdb
import ipdb
# %pdb off # desativa debug em exceção
# %pdb on  # ativa debug em exceção
# ipdb.set_trace(context=8)  para execução nesse ponto

In [22]:
def config_display():
  """
  Esta função configura as opções de display do Pandas.
  """

  # Configurando formato saída Pandas
  # define o número máximo de colunas que serão exibidas
  pd.options.display.max_columns = None

  # define a largura máxima de uma linha
  pd.options.display.width = 1000

  # define o número máximo de linhas que serão exibidas
  pd.options.display.max_rows = 100

  # define o número máximo de caracteres por coluna
  pd.options.display.max_colwidth = 50

  # se deve exibir o número de linhas e colunas de um DataFrame.
  pd.options.display.show_dimensions = True

  # número de dígitos após a vírgula decimal a serem exibidos para floats.
  pd.options.display.precision = 7


In [23]:
def config_debug():
  """
  Esta função configura as opções de debug do PyTorch e dos pacotes
  transformers e datasets.
  """

  # Define opções de impressão de tensores para o modo científico
  torch.set_printoptions(sci_mode=True) 
  """
    Significa que valores muito grandes ou muito pequenos são mostrados em notação científica.
    Por exemplo, em vez de imprimir o número 0.0000012345 como 0.0000012345, 
    ele seria impresso como 1.2345e-06. Isso é útil em situações em que os valores dos tensores 
    envolvidos nas operações são muito grandes ou pequenos, e a notação científica permite 
    uma melhor compreensão dos números envolvidos.  
  """

  # Habilita detecção de anomalias no autograd do PyTorch
  torch.autograd.set_detect_anomaly(True)
  """
    Permite identificar operações que podem causar problemas de estabilidade numérica, 
    como gradientes explodindo ou desaparecendo. Quando essa opção é ativada, 
    o PyTorch verifica se há operações que geram valores NaN ou infinitos nos tensores 
    envolvidos no cálculo do gradiente. Se for detectado um valor anômalo, o PyTorch 
    interrompe a execução e gera uma exceção, permitindo que o erro seja corrigido 
    antes que se torne um problema maior.

    É importante notar que a detecção de anomalias pode ter um impacto significativo 
    no desempenho, especialmente em modelos grandes e complexos. Por esse motivo,
    ela deve ser usada com cautela e apenas para depuração.
  """

  # Configura variável de ambiente para habilitar a execução síncrona (bloqueante) das chamadas da API do CUDA.
  os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
  """
    o Python aguarda o término da execução de uma chamada da API do CUDA antes de executar a próxima chamada. 
    Isso é útil para depurar erros no código que envolve operações na GPU, pois permite que o erro seja capturado 
    no momento em que ocorre, e não depois de uma sequência de operações que pode tornar a origem do erro mais difícil de determinar.
    No entanto, é importante lembrar que esse modo de execução é significativamente mais lento do que a execução assíncrona, 
    que é o comportamento padrão do CUDA. Por isso, é recomendado utilizar esse comando apenas em situações de depuração 
    e removê-lo após a solução do problema.
  """

  # Define o nível de verbosity do pacote transformers para info
  ## transformers.utils.logging.set_verbosity_info() 
  
  
  """
    Define o nível de detalhamento das mensagens de log geradas pela biblioteca Hugging Face Transformers 
    para o nível info. Isso significa que a biblioteca irá imprimir mensagens de log informativas sobre
    o andamento da execução, tais como tempo de execução, tamanho de batches, etc.

    Essas informações podem ser úteis para entender o que está acontecendo durante a execução da tarefa 
    e auxiliar no processo de debug. É importante notar que, em alguns casos, a quantidade de informações
    geradas pode ser muito grande, o que pode afetar o desempenho do sistema e dificultar a visualização
    das informações relevantes. Por isso, é importante ajustar o nível de detalhamento de acordo com a 
    necessidade de cada tarefa.
  
    Caso queira reduzir a quantidade de mensagens, comentar a linha acima e 
      descomentar as duas linhas abaixo, para definir o nível de verbosity como error ou warning
  
    transformers.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_warning()
  """


  # Define o modo verbose do xmode, que é utilizado no debug
  ## %xmode Verbose 

  """
    Comando usado no Jupyter Notebook para controlar o modo de exibição das informações de exceções.
    O modo verbose é um modo detalhado que exibe informações adicionais ao imprimir as exceções.
    Ele inclui as informações de pilha de chamadas completa e valores de variáveis locais e globais 
    no momento da exceção. Isso pode ser útil para depurar e encontrar a causa de exceções em seu código.
    Ao usar %xmode Verbose, as informações de exceção serão impressas com mais detalhes e informações adicionais serão incluídas.

    Caso queira desabilitar o modo verbose e utilizar o modo plain, 
    comentar a linha acima e descomentar a linha abaixo:
    %xmode Plain
  """

  """
    Dica:
    1.  pdb (Python Debugger)
      Quando ocorre uma exceção em uma parte do código, o programa para a execução e exibe uma mensagem de erro 
      com informações sobre a exceção, como a linha do código em que ocorreu o erro e o tipo da exceção.

      Se você estiver depurando o código e quiser examinar o estado das variáveis ​​e executar outras operações 
      no momento em que a exceção ocorreu, pode usar o pdb (Python Debugger). Para isso, é preciso colocar o comando %debug 
      logo após ocorrer a exceção. Isso fará com que o programa pare na linha em que ocorreu a exceção e abra o pdb,
      permitindo que você explore o estado das variáveis, examine a pilha de chamadas e execute outras operações para depurar o código.


    2. ipdb
      O ipdb é um depurador interativo para o Python que oferece recursos mais avançados do que o pdb,
      incluindo a capacidade de navegar pelo código fonte enquanto depura.
      
      Você pode começar a depurar seu código inserindo o comando ipdb.set_trace() em qualquer lugar do 
      seu código onde deseja pausar a execução e começar a depurar. Quando a execução chegar nessa linha, 
      o depurador entrará em ação, permitindo que você examine o estado atual do seu programa e execute 
      comandos para investigar o comportamento.

      Durante a depuração, você pode usar comandos:
        next (para executar a próxima linha de código), 
        step (para entrar em uma função chamada na próxima linha de código) 
        continue (para continuar a execução normalmente até o próximo ponto de interrupção).

      Ao contrário do pdb, o ipdb é um depurador interativo que permite navegar pelo código fonte em que
      está trabalhando enquanto depura, permitindo que você inspecione variáveis, defina pontos de interrupção
      adicionais e até mesmo execute expressões Python no contexto do seu programa.
  """


In [24]:
config_display()

In [25]:
config_debug()

## Rastro (neptune.ai)

Gerado rastro da execução no Neptune (detalhes no artigo [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664))


### Importação de libraries para Rastro

In [26]:
import getpass, copy, tempfile, re

In [27]:
os.environ['NEPTUNE_ALLOW_SELF_SIGNED_CERTIFICATE'] = 'TRUE'
os.environ['NEPTUNE_PROJECT'] = 'marcusborela/IA386DD'

In [28]:
os.environ['NEPTUNE_API_TOKEN'] = getpass.getpass('Informe NEPTUNE_API_TOKEN')


In [29]:
tag_contexto_rastro = 'Aula 7 - DPR'

In [30]:
neptune_version = 0

### Código Rastro

Busca implementar o rastro proposto em [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664), autores Marcus Vinícius Borela de Castro e Remis Balaniuk, com o apoio da [solução Neptune](https://app.neptune.ai/)



In [31]:
def converte_optimizer_state_dict(parm_optimizer)-> dict:
  """
    Recebe um objeto "parm_optimizer" que é do tipo "torch.optim.Optimizer" e retorna um dicionário 
    com informações sobre o otimizador.

    O dicionário de retorno é gerado a partir do estado do otimizador que é extraído da propriedade
    "state_dict()" do objeto "parm_optimizer", seu primeiro grupo de parâmetros do otimizador.
  """
  # return str(hparam['optimizer'])
  return parm_optimizer.state_dict()['param_groups'][0]

In [32]:
if neptune_version == 0:
  import neptune.new as neptune  
  class NeptuneRastroRun():
      se_geracao_rastro = True 
      neptune_project = ""
      tag_contexto_rastro = ""
      neptune_api_token = ""

      def __init__(self, parm_params:dict,  parm_lista_tag:list = None):
        # print(f"NeptuneRastroRun.init: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_params `{parm_params} ")
        if self.__class__.se_geracao_rastro:      
          self.run_neptune = neptune.init(project=self.__class__.neptune_project, api_token=self.__class__.neptune_api_token, capture_hardware_metrics=True)
          self.run_neptune['sys/name'] = self.__class__.tag_contexto_rastro
          vparams = copy.deepcopy(parm_params)
          if "optimizer" in vparams:
            vparams["optimizer"] = converte_optimizer_state_dict(vparams["optimizer"])
          if 'criterion'  in vparams:
            vparams["criterion"] = str(vparams["criterion"])
          if 'scheduler'  in vparams:
            vparams["scheduler"] = str(type(vparams["scheduler"]))
          if 'device' in vparams:
            vparams['device'] = str(vparams["device"])
          self.device = vparams["device"]
          for tag in parm_lista_tag:
            self.run_neptune['sys/tags'].add(tag)
          self.run_neptune['parameters'] = vparams
          self.tmpDir = tempfile.mkdtemp()

      @property
      def run():
        return self.run_neptune

      @classmethod
      def ativa_geracao_rastro(cls):
        cls.se_geracao_rastro = True      

      @classmethod
      def def_contexto(cls):
        cls.se_geracao_rastro = True      

      @classmethod
      def desativa_geracao_rastro(cls):
        cls.se_geracao_rastro = False      

      @classmethod
      def retorna_status_geracao_rastro(cls):
        return cls.se_geracao_rastro      

      @classmethod
      def retorna_tag_contexto_rastro(cls):
        return cls.tag_contexto_rastro 

      @classmethod
      def inicia_contexto(cls, neptune_project, tag_contexto_rastro, neptune_api_token):
        assert '.' not in tag_contexto_rastro, "NeptuneRastroRun.init(): tag_contexto_rastro não pode possuir ponto, pois será usado para gravar nome de arquivo"      
        cls.neptune_api_token = neptune_api_token
        cls.tag_contexto_rastro = tag_contexto_rastro
        cls.neptune_project = neptune_project

      def salva_metrica(self, parm_metricas={}):
        #print(f"NeptuneRastroRun.salva_metrica: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_metricas:{parm_metricas} ")
        if self.__class__.se_geracao_rastro:
          for metrica, valor in parm_metricas.items(): 
            self.run_neptune[metrica].log(valor)
  
      def gera_grafico_modelo(self, loader_train, model):
        if self.__class__.se_geracao_rastro: 
          # efetuar um forward 
          """
          se dataloader devolver x e y:
          """
          x_, y_ = next(iter(loader_train))
          x_ = x_.to(self.device)
          outputs = model(x_)
          """
          # se dataloader devolver dict:
          dados_ = next(iter(loader_train))
          outputs = model(dados_['x'].to(self.device))
          #outputs = model(x_['input_ids'].to(self.device), x_['attention_mask'].to(self.device))
          """
          nome_arquivo = os.path.join(self.tmpDir, "modelo "+ self.__class__.tag_contexto_rastro + time.strftime("%Y-%b-%d %H:%M:%S"))
          make_dot(outputs, params=dict(model.named_parameters()), show_attrs=True, show_saved=True).render(nome_arquivo, format="png")
          self.run_neptune["parameters/model_graph"].upload(nome_arquivo+'.png')
          self.run_neptune['parameters/model'] = re.sub('<bound method Module.state_dict of ', '',str(model.state_dict))      



      def stop(self):
        if self.__class__.se_geracao_rastro:         
          self.run_neptune.stop()


In [33]:
if neptune_version == 1:
  import neptune
  class NeptuneRastroRun():
      """
        Classe para geração de rastro de experimento utilizando a ferramenta Neptune.

        Busca implementar o rastro proposto em [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664),
        autores Marcus Vinícius Borela de Castro e Remis Balaniuk, com o apoio da [solução Neptune](https://app.neptune.ai/)

        Attributes:
        -----------
        se_geracao_rastro : bool
            Indica se deve ser gerado rastro de experimento. 
        neptune_project : str
            Nome do projeto criado no Neptune. 
        tag_contexto_rastro : str
            Nome da tag utilizada para identificar o experimento.
        neptune_api_token : str
            Token utilizado para autenticação na API do Neptune. 
        run_neptune : object
            Objeto que representa o experimento no Neptune.
        device : str
            Dispositivo utilizado para o treinamento do modelo.
        tmpDir : str
          Diretório temporário utilizado para salvar gráfico do modelo.          
      """
      se_geracao_rastro = True 
      neptune_project = ""
      tag_contexto_rastro = ""
      neptune_api_token = ""

      def __init__(self, parm_params:dict,  parm_lista_tag:list = None):
        """
          Método construtor da classe NeptuneRastroRun.
          
          Args:
          - parm_params: dicionário contendo os parâmetros do modelo.
          - parm_lista_tag: lista contendo tags adicionais para o experimento.
        """      
        # print(f"NeptuneRastroRun.init: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_params `{parm_params} ")
        if self.__class__.se_geracao_rastro:      
          self.run_neptune = neptune.init_run(project=self.__class__.neptune_project, api_token=self.__class__.neptune_api_token, capture_hardware_metrics=True)
          self.run_neptune['sys/name'] = self.__class__.tag_contexto_rastro
          vparams = copy.deepcopy(parm_params)
          if "optimizer" in vparams:
            vparams["optimizer"] = converte_optimizer_state_dict(vparams["optimizer"])
          if 'criterion'  in vparams:
            vparams["criterion"] = str(vparams["criterion"])
          if 'scheduler'  in vparams:
            vparams["scheduler"] = str(type(vparams["scheduler"]))
          if 'device' in vparams:
            vparams['device'] = str(vparams["device"])
          self.device = vparams["device"]
          for tag in parm_lista_tag:
            self.run_neptune['sys/tags'].add(tag)
          self.run_neptune['parameters'] = vparams
          # self.tmpDir = tempfile.mkdtemp()

      @property
      def run():
        """
        Retorna a instância do objeto run_neptune.
        """      
        return self.run_neptune

      @classmethod
      def ativa_geracao_rastro(cls):
        """
        Ativa a geração de rastro.
        """      
        cls.se_geracao_rastro = True      

      @classmethod
      def def_contexto(cls):
        """
        Define o contexto para a geração de rastro.
        """      
        cls.se_geracao_rastro = True      

      @classmethod
      def desativa_geracao_rastro(cls):
        """
        Desativa a geração de rastro.
        """      
        cls.se_geracao_rastro = False      

      @classmethod
      def retorna_status_geracao_rastro(cls):
        """
          Retorna o status da geração de rastro.
          
          Returns:
          - True se a geração de rastro está ativada, False caso contrário.
        """      
        return cls.se_geracao_rastro      

      @classmethod
      def retorna_tag_contexto_rastro(cls):
        """
          Retorna a tag do contexto de rastro.
        """      
        return cls.tag_contexto_rastro 

      @classmethod
      def inicia_contexto(cls, neptune_project, tag_contexto_rastro, neptune_api_token):
        """
        Inicia o contexto de execução no Neptune.

        Args:
            neptune_project (str): Nome do projeto no Neptune.
            tag_contexto_rastro (str): Tag que identifica o contexto de execução no Neptune.
            neptune_api_token (str): Token de acesso à API do Neptune.

        Raises:
            AssertionError: Caso a tag_contexto_rastro possua um ponto (.), 
              o que pode gerar erros na gravação de arquivo.
        """      
        assert '.' not in tag_contexto_rastro, "NeptuneRastroRun.init(): tag_contexto_rastro não pode possuir ponto, pois será usado para gravar nome de arquivo"      
        cls.neptune_api_token = neptune_api_token
        cls.tag_contexto_rastro = tag_contexto_rastro
        cls.neptune_project = neptune_project

      def salva_metrica(self, parm_metricas={}):
        """
          Salva as métricas no Neptune Run caso a geração de rastro esteja ativa.

          Parameters
          ----------
          parm_metricas: dict
              Dicionário contendo as métricas a serem salvas. As chaves devem ser os nomes das métricas e os valores devem ser
              os valores das métricas.
        """
        #print(f"NeptuneRastroRun.salva_metrica: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_metricas:{parm_metricas} ")
        if self.__class__.se_geracao_rastro:
          for metrica, valor in parm_metricas.items(): 
            self.run_neptune[metrica].append(valor)
  
      def gera_grafico_modelo(self, loader_train, model):
        """
          Gera um gráfico do modelo e o envia para o Neptune. 
          Para gerar o gráfico, um forward pass é realizado em um batch de exemplos 
          de treino e o resultado é renderizado como um gráfico de nós conectados. 
          O gráfico é salvo em um arquivo .png e enviado para o Neptune como um arquivo anexo.

          Args:
              loader_train (torch.utils.data.DataLoader): DataLoader do conjunto de treinamento.
              model (torch.nn.Module): Modelo a ser visualizado.
          
          Pendente:
            Evolui para usar from io import StringIO (buffer = io.StringIO()) ao invés de tempdir 
        """    
        return

        """
        falta ajustar make_dot
        if self.__class__.se_geracao_rastro: 
          # efetuar um forward 
          batch = next(iter(loader_train))
          # falta generalizar linha abaixo. Criar função que recebe modelo e batch como parâmetro?
          outputs = model(input_ids=batch['input_ids'].to(hparam['device']), attention_mask=batch['attention_mask'].to(hparam['device']), token_type_ids=batch['token_type_ids'].to(hparam['device']), labels=batch['labels'].to(hparam['device']))
          nome_arquivo = os.path.join(self.tmpDir, "modelo "+ self.__class__.tag_contexto_rastro + time.strftime("%Y-%b-%d %H:%M:%S"))
          make_dot(outputs, params=dict(model.named_parameters()), show_attrs=True, show_saved=True).render(nome_arquivo, format="png")
          self.run_neptune["parameters/model_graph"].upload(nome_arquivo+'.png')
          self.run_neptune['parameters/model'] = re.sub('<bound method Module.state_dict of ', '',str(model.state_dict))      
        """


      def stop(self):
        """
          Para a execução do objeto Neptune. Todos os experimentos do Neptune são sincronizados com o servidor, e nenhum outro 
          experimento poderá ser adicionado a este objeto após a chamada a este método.
        """
        if self.__class__.se_geracao_rastro:         
          self.run_neptune.stop()


### Definindo parâmetros para o rastro



In [34]:
NeptuneRastroRun.inicia_contexto(os.environ['NEPTUNE_PROJECT'], tag_contexto_rastro,  os.environ['NEPTUNE_API_TOKEN'])
#NeptuneRastroRun.desativa_geracao_rastro()

# Carregando modelo e tokenizador

In [35]:
MODEL_NAME = 'microsoft/MiniLM-L12-H384-uncased'

# NOME_CAMINHO_MODELO = "/home/borela/fontes/deep_learning_em_buscas_unicamp/modelo/" + MODEL_NAME
# assert os.path.exists(NOME_CAMINHO_MODELO), f"Path para {NOME_CAMINHO_MODELO} não existe!"


In [36]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [37]:
# Se tiver que treinar os modelos, abre
models = {'query': AutoModel.from_pretrained(MODEL_NAME).to(hparam['device']),
'passage' : AutoModel.from_pretrained(MODEL_NAME).to(hparam['device'])}

In [38]:
print(models['query'].config.max_position_embeddings, models['passage'].config.vocab_size)
# 512 e 30522
#models['query'].config.__dict__


512 30522


# Carga dos dados msmarco_triples.train.tiny.tsv

In [39]:
if not os.path.exists(f"{DIRETORIO_TRABALHO}/msmarco_triples.train.tiny.tsv"):
    !wget https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv
    !mv msmarco_triples.train.tiny.tsv {DIRETORIO_TRABALHO}
    print("Pasta criada!")
else:
    print("Pasta já existia!")

Pasta já existia!


In [40]:
df = pd.read_csv(f"{DIRETORIO_TRABALHO}/msmarco_triples.train.tiny.tsv", delimiter="\t", 
                 header=None, names=["query", "positive", "negative"])

In [41]:
print(df.shape)

(11000, 3)


In [42]:
print(df.head())

                                            query                                           positive                                           negative
0        is a little caffeine ok during pregnancy  We donât know a lot about the effects of caf...  It is generally safe for pregnant women to eat...
1               what fruit is native to australia  Passiflora herbertiana. A rare passion fruit n...  The kola nut is the fruit of the kola tree, a ...
2              how large is the canadian military  The Canadian Armed Forces. 1  The first large-...  The Canadian Physician Health Institute (CPHI)...
3                            types of fruit trees  Cherry. Cherry trees are found throughout the ...  The kola nut is the fruit of the kola tree, a ...
4  how many calories a day are lost breastfeeding  Not only is breastfeeding better for the baby,...  However, you still need some niacin each day; ...

[5 rows x 3 columns]


Verificando correção do arquivo!

In [43]:
print(df.isnull().sum())

query       0
positive    0
negative    0
Length: 3, dtype: int64


In [44]:
# !pip install -q ftfy
import ftfy

In [45]:
x = 'We donât know a lot about the effects'

In [46]:
print(ftfy.fix_text(x))

We don't know a lot about the effects


In [47]:
df.applymap(len).mean()

query        34.2256364
positive    353.7535455
negative    340.4646364
Length: 3, dtype: float64

In [48]:
# pois treinaremos doc2query apenas para geração de queries relevantes
del df['negative']

In [49]:
df['query'] = df['query'].apply(ftfy.fix_text)
df['positive'] = df['positive'].apply(ftfy.fix_text)


In [50]:
df.head()

,query,positive
0,is a little caffeine ok during pregnancy,We don't know a lot about the effects of caffe...
1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...
2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...
3,types of fruit trees,Cherry. Cherry trees are found throughout the ...
4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,..."


# Divisão em treino e validação

In [51]:
from sklearn.model_selection import train_test_split

In [52]:
passage_train, passage_valid, query_train, query_valid = train_test_split(df['positive'].tolist(), 
                                                      df['query'].tolist(),
                                                      test_size=1000, 
                                                      random_state=num_semente)

In [53]:
print(type(query_valid), len(query_valid), query_valid[0])

<class 'list'> 1000 how many fitbit steps equal a mile


In [54]:
print(type(passage_train), len(passage_train), passage_train[0])

<class 'list'> 10000 There are 40 weeks in a school year - and 12 weeks of holidays. Unless you live somewhere snowy and you have to take snow days. These can extend the length by another week or so. There are 40 weeks in a school year - and 12 weeks of holidays. Unless you live somewhere snowy and you have to take snow days.


# Criando dataset

In [55]:
from scipy import stats

In [56]:
class DprDataset(torch.utils.data.Dataset):
    """
      Classe para criar um dataset de texto e query.
    """  
    def __init__(self, passages: list, queries:list, tokenizer):
      """
      Inicializa um novo objeto DprDataset.

      Args:
          passages (np.ndarray): um array com as strings de passage.
          passages (np.ndarray): um array com as strings de queries.
          tokenizer: um objeto tokenizer do Hugging Face Transformers.
      Raises:
          AssertionError: se os parâmetros não estiverem no formato esperado.
      """
      # Verifica se os parâmetros são do tipo esperado
      # assert isinstance(passages, np.ndarray), f"Parâmetro passages deve ser do tipo np.ndarray e não {type(passages)}"
      # assert isinstance(queries, np.ndarray), f"Parâmetro queries deve ser do tipo np.ndarray e não {type(queries)}"
      for row in passages:
          assert isinstance(row, str), f"Each element in passages.row must be a string e não {type(row)}"
          break

            # Salvar os dados dos tensores para adiantar o tempo de processamento
      self.tokenized_passages = tokenizer(passages, return_length=True)
      self.tokenized_queries = tokenizer(queries, return_length=True)
      
      print("tokenized_passages size stats:\n{}\n".format(stats.describe(self.tokenized_passages['length'])))
      print("tokenized_queries size stats:\n{}\n".format(stats.describe(self.tokenized_queries['length']))) 

      self.shuffle()


    def shuffle(self):
        # Fonte: colega Eduardo Seiti
        self.samples_order = list(range(len(self.tokenized_queries['input_ids'])))
        np.random.shuffle(self.samples_order)


    def __len__(self):
        return len(self.tokenized_queries['input_ids'])


    def __getitem__(self, index):
        return {'passage': {'input_ids': self.tokenized_passages['input_ids'][self.samples_order[index]],
                            'attention_mask': self.tokenized_passages['attention_mask'][self.samples_order[index]]},
                'query' : {'input_ids': self.tokenized_queries['input_ids'][self.samples_order[index]],
                            'attention_mask': self.tokenized_queries['attention_mask'][self.samples_order[index]]}}


In [57]:
from transformers import  BatchEncoding

In [58]:
class DPRCollator(object):
    # Fonte: colega Eduardo Seiti
    def __init__(self, type='passage', tokenizer=None):
        assert type in ('passage', 'query'), f"type {type} deve estar em ('passage', 'query')"
        self.type = type
        self.tokenizer = tokenizer


    def __call__(self, batch):
        # print(f"batch {batch}")
        lista = [item[self.type] for item in batch]
        # print(f"lista {lista}")
        padded_batch = self.tokenizer.pad(lista, return_tensors='pt')
        # print(f"padded_batch {padded_batch}")
        return BatchEncoding(padded_batch)

#### Testando o MyDataset e o Dataloader

In [59]:
# Cria dados fictícios
texts = ['This is the first text',
                  'This is text 2.1',
                  'This is text 3.1',
                  'This is text 4.1',
                  'This is text 5.1',
                  'This is text 6.1',
                  'This is text 7.1']
queries = ['This is the first query',
                  'This is query 2.1',
                  'This is query 3.1',
                  'This is query 4.1',
                  'This is query 5.1',
                  'This is query 6.1',
                  'This is query 7.1']

In [60]:
print(tokenizer(texts))

{'input_ids': [[101, 2023, 2003, 1996, 2034, 3793, 102], [101, 2023, 2003, 3793, 1016, 1012, 1015, 102], [101, 2023, 2003, 3793, 1017, 1012, 1015, 102], [101, 2023, 2003, 3793, 1018, 1012, 1015, 102], [101, 2023, 2003, 3793, 1019, 1012, 1015, 102], [101, 2023, 2003, 3793, 1020, 1012, 1015, 102], [101, 2023, 2003, 3793, 1021, 1012, 1015, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}


In [61]:
# Cria um objeto da classe MyDataset
dummy_dataset = DprDataset(passages=texts, queries=queries, tokenizer=tokenizer)

tokenized_passages size stats:
DescribeResult(nobs=7, minmax=(7, 8), mean=7.857142857142857, variance=0.1428571428571429, skewness=-2.041241452319312, kurtosis=2.1666666666666563)

tokenized_queries size stats:
DescribeResult(nobs=7, minmax=(7, 8), mean=7.857142857142857, variance=0.1428571428571429, skewness=-2.041241452319312, kurtosis=2.1666666666666563)



In [62]:
# Testa o método __len__()
assert len(dummy_dataset) == 7


In [63]:

# Testa o método __getitem__()
sample = dummy_dataset[0]


In [64]:
print(sample.keys())

dict_keys(['passage', 'query'])


In [65]:
print(sample['passage'])

{'input_ids': [101, 2023, 2003, 3793, 1016, 1012, 1015, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [66]:

assert set(sample.keys()) == {'passage', 'query'} # 
assert set(sample['passage']) == {'input_ids', 'attention_mask'} # 
assert set(sample['query']) == {'input_ids', 'attention_mask'} # 
assert isinstance(sample['passage']['input_ids'], list)


In [67]:
print(sample)

{'passage': {'input_ids': [101, 2023, 2003, 3793, 1016, 1012, 1015, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}, 'query': {'input_ids': [101, 2023, 2003, 23032, 1016, 1012, 1015, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}}


In [68]:
from torch.utils.data import DataLoader

In [69]:
dummy_loader = DataLoader(dummy_dataset, batch_size=3, shuffle=False, num_workers=0, 
                                                        collate_fn=DPRCollator('passage', tokenizer))

In [70]:
first_batch = next(iter(dummy_loader))

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [71]:
print(first_batch)

{'input_ids': tensor([[ 101, 2023, 2003, 3793, 1016, 1012, 1015,  102],
        [ 101, 2023, 2003, 3793, 1018, 1012, 1015,  102],
        [ 101, 2023, 2003, 3793, 1019, 1012, 1015,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]])}


# Criando datasets e dataloaders

In [72]:
# train_dataset = MyDataset(X_train, Y_train, tokenizer)
valid_dataset = DprDataset(passages=passage_valid, queries= query_valid, tokenizer= tokenizer)

tokenized_passages size stats:
DescribeResult(nobs=1000, minmax=(19, 270), mean=80.419, variance=1082.6901291291294, skewness=1.2249734252598994, kurtosis=1.8839424951259751)

tokenized_queries size stats:
DescribeResult(nobs=1000, minmax=(4, 32), mean=9.072, variance=7.095911911911913, skewness=1.5337855672942304, kurtosis=6.957955492738629)



In [73]:
train_dataset = DprDataset(passages=passage_train, queries=query_train, tokenizer=tokenizer)

tokenized_passages size stats:
DescribeResult(nobs=10000, minmax=(13, 280), mean=80.6696, variance=1047.7126071007099, skewness=1.126008868700056, kurtosis=1.4884134181168802)

tokenized_queries size stats:
DescribeResult(nobs=10000, minmax=(4, 43), mean=9.1068, variance=8.229416701670168, skewness=2.2592354361208167, kurtosis=14.94412756929329)



In [74]:
print(len(train_dataset),len(valid_dataset))

10000 1000


# Teste do modelo

In [75]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 67.35
 available 56.04
 used 10.19
 free 11.01
 cached 45.64
 buffers 0.5
/nGPU
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Tue Apr 18 18:19:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.39.01    Driver Version: 510.39.01    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+===================

In [76]:
models['passage'].config.pad_token_id, models['passage'].config.eos_token_id, models['passage'].config.sep_token_id, models['passage'].config.max_position_embeddings

(0, None, None, 512)

In [77]:
hparam['batch_size'] = 2
print(hparam)

{'num_workers_dataloader': 0, 'device': device(type='cuda', index=0), 'batch_size': 2}


In [78]:
sample_passage = next(iter(DataLoader(valid_dataset,batch_size=hparam['batch_size'],collate_fn=DPRCollator('passage', tokenizer))))
sample_query = next(iter(DataLoader(valid_dataset,batch_size=hparam['batch_size'],collate_fn=DPRCollator('query', tokenizer))))


In [79]:
print(sample_passage)

{'input_ids': tensor([[  101, 14925, 14399,  2594, 10032,  5158,  2012,  1037,  3446,  1997,
          2539,  1012,  1021,  3572,  2566,  1015,  1010,  2199,  3653, 28207,
          9243,  1999,  2167,  2637,  1998,  2003,  1037,  2877,  3426,  1997,
         11062, 13356,  1999,  1996,  2034, 12241, 20367,  1012,  3618,  7073,
          1997,  3891,  5876,  1998,  5301,  2974,  1006, 16012, 15869, 16387,
          1998, 11087,  3385,  9888,  1007,  3499, 14925, 14399,  2594, 10032,
          2000,  2022,  4453,  2077,  1996,  2458,  1997,  2166,  1011,  8701,
          2824,  1012,   102],
        [  101,  3361,  2490,  1011,  3361,  4219,  3024,  2000,  2191,  2070,
          2622,  2825,  1025,  1996,  3192,  3024,  2490,  2005,  1996,  7551,
          1012,  3361,  5150,  1010,  4804,  1010,  5150,  1010,  2490,  1012,
          7692,  1011,  2800,  3120,  1997,  7177,  1025,  1037,  2047,  2030,
          3914,  4425,  2008,  2064,  2022,  4567,  2588,  2043,  2734,  1012,
       

In [80]:
print(sample_passage['input_ids'].shape, len(sample_query['input_ids']))

torch.Size([2, 73]) 2


In [81]:
print(sample_query)

{'input_ids': tensor([[  101,  2129,  4703,  2515, 14925, 14399,  2594, 10032,   102],
        [  101,  9375,  3361,  2490,   102,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 0, 0, 0]])}


In [82]:
print(sample_query['input_ids'].shape, sample_passage['input_ids'].shape)

torch.Size([2, 9]) torch.Size([2, 73])


In [83]:
saida_passage = models['passage'](**sample_passage.to(hparam['device']))

In [84]:
saida_query = models['query'](**sample_query.to(hparam['device']))

In [85]:
# obtendo embeddings do token cls, o primeiro, índice 0
cls_embedding_saida_passage = saida_passage.last_hidden_state[:, 0, :]

In [86]:
# obtendo embeddings do token cls, o primeiro, índice 0
cls_embedding_saida_query = saida_query.last_hidden_state[:, 0, :]

In [87]:
print(type(cls_embedding_saida_query))

<class 'torch.Tensor'>


In [88]:
print(cls_embedding_saida_query.shape, cls_embedding_saida_passage.shape)

torch.Size([2, 384]) torch.Size([2, 384])


In [89]:
assert cls_embedding_saida_query.shape[1] == 384, "Cls[1] deveria ser do tamanho do embedding size"


In [90]:
assert cls_embedding_saida_query.shape[0] == 2, "Cls[0] deveria ser do tamanho do batch size"


In [91]:
print(models['passage'].config.pad_token_id, models['passage'].config.vocab_size)

0 30522


In [92]:
del cls_embedding_saida_query, cls_embedding_saida_passage, saida_passage, saida_query
del sample_passage, sample_query

# Loss e avaliação

In [93]:
def compute_loss(passages_outputs, topics_outputs):
    # Fonte: colega Carísio
    
    # Extrai a última camada oculta associada ao token [CLS]
    tcls_queries = topics_outputs.last_hidden_state[:, 0, :]
    tcls_docs    = passages_outputs.last_hidden_state[:, 0, :]

    # Normaliza os tensores
    #tcls_queries = tcls_queries / torch.norm(tcls_queries, dim=1, keepdim=True)
    #tcls_docs = tcls_docs / torch.norm(tcls_docs, dim=1, keepdim=True)
    
    # Agora é necessário calcular a loss. Para isso, o primeiro passo é
    # calcular a similaridade entre uma query e documento (sim(q, d))
    similaridade = torch.matmul(tcls_queries, torch.transpose(tcls_docs, 0, 1))

    # Calcula a exponencial da similaridade
    exp_sim = torch.exp(similaridade)
    
    # Calcula a loss
    # Valor para o denominador: inclui os exemplos positivos e negativos (in-batch)
    soma_linhas = exp_sim.sum(dim=1) 
    diagonal = torch.diag(exp_sim)
    log_loss = -1* torch.log(diagonal/soma_linhas)
    
    loss = torch.mean(log_loss)
    return loss    

In [94]:
def validation_step(parm_models, 
             parm_dataloaders):
    # Fonte: colega Eduardo Seiti (ajustada)
    
    eval_losses = []

    for model in parm_models:
       parm_models[model].eval()

    with torch.no_grad():
        for batch in tqdm(list(zip(parm_dataloaders['passage'], parm_dataloaders['query'])), mininterval=0.5, desc="Eval", disable=False):
            passages_outputs = parm_models['passage'](**batch[0].to(hparam['device']))
            topics_outputs = parm_models['query'](**batch[1].to(hparam['device']))
            eval_losses.append(compute_loss(passages_outputs, topics_outputs).cpu().numpy())

    final_loss = np.mean(eval_losses)
    print("Eval loss: {:0.3f}".format(final_loss))

    return final_loss


# Treinamento e Validação 



## Calculando loss

In [95]:
valid_loaders = {
     'passage': DataLoader(valid_dataset, batch_size=hparam['batch_size'], shuffle=False, 
                           drop_last=True, num_workers=hparam['num_workers_dataloader'],
                           collate_fn=DPRCollator('passage', tokenizer)),
     'query': DataLoader(valid_dataset, batch_size=hparam['batch_size'], shuffle=False, 
                           drop_last=True, num_workers=hparam['num_workers_dataloader'],
                           collate_fn=DPRCollator('query', tokenizer))}

In [96]:
validation_loss = validation_step(models, valid_loaders)

Eval: 100%|██████████| 500/500 [00:10<00:00, 48.51it/s]

Eval loss: 0.680


In [102]:
print(len(valid_loaders['passage']))

500

## Funções auxiliares de treinamento

In [97]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [112]:
def treina_modelo (parm_models, parm_loader_trains, parm_loader_valids, hparam:dict, parm_se_gera_rastro:bool=True,  parm_intervalo_print = 10):
  """
  """
  global DIRETORIO_TRABALHO, tag_contexto_rastro
  if parm_se_gera_rastro:
    rastro_neptune = NeptuneRastroRun(hparam, parm_lista_tag= tag_contexto_rastro )
  try:
    print(f'hparam: {hparam}')
    path_modelo = f'{DIRETORIO_TRABALHO}_best_model_inicio_treino_{time.strftime("%Y-%b-%d %H:%M:%S")}'
    n_examples = 0
    best_validation_loss= float('inf') # desejamos reduzir a distância entre pares (embeddings) próximos
    best_epoca = 0
    history = []
    validation_loss = validation_step(parm_models, parm_loader_valids)       
    metrica_rastro = {"valid/loss": validation_loss}  
    print(f'Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")} Métricas iniciais em validação: {metrica_rastro} Serão treinadas {hparam["max_examples"]} amostras')
    history.append(metrica_rastro)
    if parm_se_gera_rastro:
      rastro_neptune.salva_metrica(metrica_rastro)

    time_inicio_treino = time.time()
    ultima_epoca_treinada = 0
    for model in parm_models:
       parm_models[model].train()    

    best_models_dict = {}
    train_losses = []
    for epoch in tqdm(range(hparam['num_epochs']), desc='Epochs'):    
        

        ultima_epoca_treinada += 1 

        for batch in tqdm(list(zip(parm_loader_trains['passage'], parm_loader_trains['query'])), mininterval=0.5, desc="Train", disable=False):


            passages_outputs = parm_models['passage'](**batch[0].to(hparam['device']))
            topics_outputs = parm_models['query'](**batch[1].to(hparam['device']))

            loss = hparam['criterion'](passages_outputs, topics_outputs)
            
            loss.backward()

            n_examples += len(batch[0]['input_ids']) # Increment of batch size

            for model in parm_models:
              hparam[f'optimizer_{model}'].step()
              hparam[f'scheduler_{model}'].step()  
              hparam[f'optimizer_{model}'].zero_grad()

            train_losses.append(loss.detach().cpu().numpy()) 
       
        if not isinstance( parm_loader_trains['passage'], list): # caso de treino com overfit 
            parm_loader_trains['passage'].dataset.shuffle()  # já funciona pra o 'query' também

        if ultima_epoca_treinada % hparam['eval_every_epocas'] == 0:

            train_loss = np.average(train_losses)

            validation_loss = validation_step(parm_models, parm_loader_valids)                

            train_losses = []

            metrica_rastro = {"train/loss": train_loss, 
                              "train/n_examples": n_examples, 
                              "train/learning_rate": hparam["optimizer_passage"].param_groups[0]["lr"],
                              "valid/loss": validation_loss}  
            history.append(metrica_rastro)
            if parm_se_gera_rastro:
              rastro_neptune.salva_metrica(metrica_rastro)

            sufixo_msg = ""
            
            # Salvando o melhor modelo de acordo com a loss de validação
            if validation_loss < best_validation_loss:
                best_models_dict = {'passage': parm_models['passage'].state_dict(),
                    'query': parm_models['query'].state_dict(),
                }
                best_validation_loss= validation_loss
                best_epoca = epoch
                sufixo_msg += f" nova best epoca {validation_loss}"

                # salva quando encontrado best_epoca
                # se não houve melhoria em 1 epoca anterior
                
                # if qtd_metrica_sem_melhor_metrica >= 1:                    
                #  torch.save(parm_model, path_modelo)    
                #  sufixo_msg += f"; modelo salvo em {path_modelo}"
                #  print(sufixo_msg)

                qtd_metrica_sem_melhor_metrica = 0
              
                # print('best model')
            elif hparam['early_stop'] <= (ultima_epoca_treinada - best_epoca):
                print(f"Parando por critério de early_stop no step {ultima_epoca_treinada} sendo best_epoca {best_epoca} e ealy_stop {hparam['early_stop']}")
                break
            else:
                qtd_metrica_sem_melhor_metrica +=1

            if parm_intervalo_print > 0:
                if ultima_epoca_treinada%parm_intervalo_print == 0: 
                  print(f'Época: {ultima_epoca_treinada} Amostras:{n_examples:d} de um total de {int(hparam["max_examples"])} ({100*n_examples/hparam["max_examples"]:.3f}%)')
                  print(f'Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")} lr: {hparam["optimizer_passage"].param_groups[0]["lr"]:.5e} Treino loss: {train_loss:.4f} Validação loss: {validation_loss:.4f} {sufixo_msg}')

            for model in parm_models:
              parm_models[model].train() 

            
    # calculando tempo gasto e médio por step
    tempo_treino = time.time() - time_inicio_treino   
    print(f"Tempo gasto total {tempo_treino:9.5f}, steps: {ultima_epoca_treinada}, tempo por step {tempo_treino/ultima_epoca_treinada:9.5f}")
    
    print(f'Final: Step: {ultima_epoca_treinada} Amostras:{n_examples:d}  {100*n_examples/hparam["max_examples"]:.3f}%  Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")} lr:{hparam["optimizer_passage"].param_groups[0]["lr"]:.5e} Train loss: {train_loss:.4f}  Validação loss: {validation_loss:.4f} ')

    if 'passage' in best_models_dict:  # pode não existir se treino de overfit
      for model in parm_models:
          parm_models[model].load_state_dict(best_models_dict[model])
          torch.save(parm_models[model], f"{path_modelo}_{model}.pt")    
      print(f"Modelo com melhor resultado em validação (step {best_epoca}) salvo após treino em {path_modelo}")


    validation_loss = validation_step(parm_models, parm_loader_valids)   

    metrica_rastro = {"valid/loss": validation_loss}      
    print(f" Resultado com dados de teste para modelo treinado: {metrica_rastro}")
    if parm_se_gera_rastro:
      rastro_neptune.run_neptune["context/tempo_treino"] = tempo_treino
      rastro_neptune.run_neptune["context/tempo_treino_por_step"] = tempo_treino/ultima_epoca_treinada
      rastro_neptune.run_neptune["valid/best_epoca"] = best_epoca
      rastro_neptune.salva_metrica(metrica_rastro)
      #rastro_neptune.gera_grafico_modelo(parm_loader_train, parm_model)    


  finally:
    if parm_se_gera_rastro:
      rastro_neptune.stop()


  return {"loss_validacao":validation_loss, "loss_treino":train_loss, "best_validation_loss":best_validation_loss,  "best_epoca": best_epoca} #, "best_model_dict": best_model_dict}

Limpa o cache da memória da GPU
torch.cuda.empty_cache()

In [99]:
from transformers import AdamW #get_cosine_with_hard_restarts_schedule_with_warmup


In [100]:
from torch.optim.lr_scheduler import LambdaLR


In [101]:

def linear_warmup_cosine_annealing_lr(optimizer, num_warmup_steps, num_total_steps, min_lr, max_lr):
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda current_step: lr_lambda(current_step, num_warmup_steps, num_total_steps, min_lr, max_lr))
    # print(f"criado scheduler com num_warmup_steps {num_warmup_steps}, num_total_steps {num_total_steps}, min_lr {min_lr}, max_lr {max_lr}")
    return scheduler

def lr_lambda(current_step, num_warmup_steps, num_total_steps, min_lr, max_lr):
    if current_step < num_warmup_steps:
        val = min(max_lr, float(current_step) / float(max(1, num_warmup_steps)))
        # print(f"Em lr_lambda current_step {current_step} < num_warmup_steps {num_warmup_steps} val {val}")
        return val
    else:
        progress = float(current_step - num_warmup_steps) / float(max(1, num_total_steps - num_warmup_steps))
        val = max(min_lr, 0.5 * (1.0 + math.cos(math.pi * progress)))
        # print(f"Em lr_lambda current_step {current_step} >= num_warmup_steps {num_warmup_steps} val {val}")
        return val


In [120]:
def ajusta_parametro_grid(hparam, combinacao_parametro, parm_models, se_treina_poucos_dados:bool=False):
  parametro_esperado_grid = ("batch_size", 
                             "num_epochs", "learning_rate")
  if not parm_models or not isinstance(parm_models,dict):
    raise Exception("Necessário informar model (tipo dict)!")                            
  if 'passage' not in parm_models or 'query' not in parm_models:
    raise Exception("Necessário informar model!")                            
  for nome_parametro in parametro_esperado_grid:
      if nome_parametro not in combinacao_parametro:
          raise NotImplementedError(f'Gride de parâmetros está incompleto, não contem {nome_parametro}')
      hparam[nome_parametro] = combinacao_parametro[nome_parametro]
  for nome_parametro in combinacao_parametro:
      if nome_parametro not in parametro_esperado_grid:
          raise NotImplementedError(f'Gride de parâmetros está com parâmetro adicional não tratado: {nome_parametro}')
      hparam[nome_parametro] = combinacao_parametro[nome_parametro]
  hparam['num_workers_dataloader'] = 0 # MUDAR DEPOIS

  # The drop_last=True parameter ignores the last batch 
  # (when the number of examples in your dataset is not divisible by your batch_size ) 
  hparam['drop_last'] = True
  train_loaders = {
     'passage': DataLoader(train_dataset, batch_size=hparam['batch_size'], shuffle=False, 
                           drop_last=hparam['drop_last'], num_workers=hparam['num_workers_dataloader'],
                           collate_fn=DPRCollator('passage', tokenizer)),
     'query': DataLoader(train_dataset, batch_size=hparam['batch_size'], shuffle=False, 
                           drop_last=hparam['drop_last'], num_workers=hparam['num_workers_dataloader'],
                           collate_fn=DPRCollator('query', tokenizer))}
                      
  valid_loaders = {
     'passage': DataLoader(valid_dataset, batch_size=hparam['batch_size'], shuffle=False, 
                           drop_last=hparam['drop_last'], num_workers=hparam['num_workers_dataloader'],
                           collate_fn=DPRCollator('passage', tokenizer)),
     'query': DataLoader(valid_dataset, batch_size=hparam['batch_size'], shuffle=False, 
                           drop_last=hparam['drop_last'], num_workers=hparam['num_workers_dataloader'],
                           collate_fn=DPRCollator('query', tokenizer))}
                      
  hparam['train_size'] = len(train_dataset) 
  hparam['valid_size'] = len(valid_dataset) 
  if se_treina_poucos_dados:
    train_loaders = {
        'passage': [next(iter(train_loaders['passage']))],
        'query': [next(iter(train_loaders['query']))] } # para overfit com poucos dados (1 batch)
  hparam['steps_train_per_epoch'] = len(train_loaders['passage'])

  hparam['max_examples'] = hparam['num_epochs'] * hparam['train_size']
  hparam['eval_every_epocas'] = 1 
  hparam['max_steps'] = hparam['num_epochs'] * hparam['steps_train_per_epoch']   # (hparam['train_size'] / hparam['batch_size'] )

  hparam['early_stop'] = 10 * hparam['eval_every_epocas']
  hparam['criterion'] = compute_loss
  
  inicializa_seed(123)
  
  hparam['num_warmup_steps'] = int(hparam['max_steps'] * 0.1) 

  hparam['num_params'] = count_parameters(parm_models['passage'])
  print(f"Number of model parameters: {hparam['num_params']}")
  # hparam['learning_rate'] =  3e-5 # 1e-3

  for model in parm_models:
    hparam[f'optimizer_{model}'] = AdamW(parm_models[model].parameters(), lr=hparam['learning_rate'])
    #hparam[f'scheduler_{model}'] = get_cosine_with_hard_restarts_schedule_with_warmup(hparam[f'optimizer_{model}'], num_warmup_steps=hparam['num_warmup_steps'],num_training_steps=hparam['max_steps'],) 
    hparam[f'scheduler_{model}'] = linear_warmup_cosine_annealing_lr(optimizer=hparam[f'optimizer_{model}'],   num_warmup_steps = hparam['num_warmup_steps'], num_total_steps= hparam['max_steps'], min_lr=1e-7,  max_lr=0.001 )  # Criação do scheduler linear com warm-up e decaimento[
  
  return hparam, parm_models, train_loaders, valid_loaders

In [121]:
def treina_grid(hparam, gridparam, parm_models, parm_se_gera_rastro:bool=True, se_treina_poucos_dados:bool=False): 
  if not parm_models or not isinstance(parm_models,dict):
    raise Exception("Necessário informar model (tipo dict)!")                            
  if 'passage' not in parm_models or 'query' not in parm_models:
    raise Exception("Necessário informar model!")                               

  keys, values = zip(*gridparam.items())
  lista_combinacao_grid = [dict(zip(keys, v)) for v in itertools.product(*values)]  
  total_combinacao = len(lista_combinacao_grid)
  print(f"Serão {total_combinacao} experimentações de parâmetros")
  qtd_experimento = 1
  lista_resultado = []
  # for cnt_combinacao, combinacao in enumerate(tqdm(lista_combinacao_grid, desc=f"Experimento {qtd_experimento}/{total_combinacao}")):
  for cnt_combinacao, combinacao in enumerate(lista_combinacao_grid):
    print(f"\n\nNUM: {qtd_experimento}/{total_combinacao} : {combinacao} ")
    hparam, parm_models, train_loaders, valid_loaders = ajusta_parametro_grid(hparam, combinacao, parm_models, se_treina_poucos_dados=se_treina_poucos_dados)
    #ipdb.set_trace(context=4)
    resultado = treina_modelo(parm_models, parm_loader_trains=train_loaders, parm_loader_valids=valid_loaders,
                          hparam=hparam,
                          parm_se_gera_rastro=parm_se_gera_rastro,  parm_intervalo_print=1)
    
    qtd_experimento += 1
    print(cnt_combinacao, resultado)
    lista_resultado.append((cnt_combinacao, resultado))
  return lista_resultado


# Validando configuração de treinamento

### Testando em poucos dados (Overfit)

In [134]:
# Se tiver que treinar os modelos, abre
models = {'query': AutoModel.from_pretrained(MODEL_NAME).to(hparam['device']),
'passage' : AutoModel.from_pretrained(MODEL_NAME).to(hparam['device'])}

In [135]:
gridparam = { 
               'learning_rate': [ 1e-4],
               'num_epochs':[10],
               'batch_size':[8],
             }                           

In [136]:
resultado = treina_grid(hparam, gridparam, models, parm_se_gera_rastro = False, se_treina_poucos_dados=True)

Serão 1 experimentações


NUM: 1/1 : {'learning_rate': 0.0001, 'num_epochs': 10, 'batch_size': 8} 
Number of model parameters: 33360000
hparam: {'num_workers_dataloader': 0, 'device': device(type='cuda', index=0), 'batch_size': 8, 'num_epochs': 10, 'learning_rate': 0.0001, 'drop_last': True, 'train_size': 10000, 'valid_size': 1000, 'max_examples': 80, 'eval_every_epocas': 1, 'early_stop': 20, 'criterion': <function compute_loss at 0x7f5bd4e9d700>, 'num_warmup_steps': 8, 'num_params': 33360000, 'optimizer_query': AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    initial_lr: 0.0001
    lr: 0.0
    weight_decay: 0.0
), 'scheduler_query': <torch.optim.lr_scheduler.LambdaLR object at 0x7f5bd49ff790>, 'optimizer_passage': AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    initial_lr: 0.0001
    lr: 0.0
    weight_decay: 0.0
), 'scheduler_passage': <torch.optim.lr_scheduler.LambdaLR object at 0x7f5af22ff7c0>}


Eval: 100%|██████████| 125/125 [00:03<00:00, 38.99it/s]


Eval loss: 2.056
Momento: [2023-Apr-18 00:16:44] Métricas iniciais em validação: {'valid/loss': 2.0563312} Serão treinadas 80 amostras


Epochs:  10%|█         | 1/10 [00:03<00:34,  3.84s/it]

Eval loss: 2.056
Época: 1 Amostras:8 de um total de 80 (10.000%)
Momento: [2023-Apr-18 00:16:48] lr: 1.25000e-05 Treino loss: 2.0446 Validação loss: 2.0563  nova best epoca 2.056331157684326


...

Eval loss: 1.994
Época: 9 Amostras:72 de um total de 80 (90.000%)
Momento: [2023-Apr-18 00:17:18] lr: 9.99524e-05 Treino loss: 1.0864 Validação loss: 1.9944 


Epochs: 100%|██████████| 10/10 [00:37<00:00,  3.70s/it]


Eval loss: 2.092
Época: 10 Amostras:80 de um total de 80 (100.000%)
Momento: [2023-Apr-18 00:17:21] lr: 9.98097e-05 Treino loss: 0.7660 Validação loss: 2.0921 
Tempo gasto total  37.00500, steps: 10, tempo por step   3.70050
Final: Step: 10 Amostras:80  100.000%  Momento: [2023-Apr-18 00:17:21] lr:9.98097e-05 Train loss: 0.7660  Validação loss: 2.0921 
Modelo com melhor resultado em validação (step 7) salvo após treino em /home/borela/fontes/deep_learning_em_buscas_unicamp/local/dpr_best_model_inicio_treino_2023-Apr-18 00:16:41


Eval: 100%|██████████| 125/125 [00:03<00:00, 38.90it/s]

Eval loss: 2.092
 Resultado com dados de teste para modelo treinado: {'valid/loss': 2.0920947}
0 {'loss_validacao': 2.0920947, 'loss_treino': 0.76597375, 'best_validation_loss': 1.9845304, 'best_epoca': 7}


In [137]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 67.35
 available 52.03
 used 14.17
 free 23.16
 cached 29.38
 buffers 0.64
/nGPU
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Tue Apr 18 00:17:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.39.01    Driver Version: 510.39.01    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+==================

### Treinando em todos os dados (3 experimentos)

In [138]:
# Se tiver que treinar os modelos, abre
models = {'query': AutoModel.from_pretrained(MODEL_NAME).to(hparam['device']),
'passage' : AutoModel.from_pretrained(MODEL_NAME).to(hparam['device'])}

#### batch size 32 (zero_grad após chamada do modelo, antes da chamada de loss.backward)

In [139]:
gridparam = { 
               'learning_rate': [ 1e-4],
               'num_epochs':[50],
               'batch_size':[32],
             }             

In [140]:
treina_grid(hparam, gridparam, models, parm_se_gera_rastro = False, se_treina_poucos_dados=False)

Serão 1 experimentações


NUM: 1/1 : {'learning_rate': 0.0001, 'num_epochs': 50, 'batch_size': 32} 
Number of model parameters: 33360000
hparam: {'num_workers_dataloader': 0, 'device': device(type='cuda', index=0), 'batch_size': 32, 'num_epochs': 50, 'learning_rate': 0.0001, 'drop_last': True, 'train_size': 10000, 'valid_size': 1000, 'max_examples': 1600, 'eval_every_epocas': 1, 'early_stop': 20, 'criterion': <function compute_loss at 0x7f5bd4e9d700>, 'num_warmup_steps': 160, 'num_params': 33360000, 'optimizer_query': AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    initial_lr: 0.0001
    lr: 0.0
    weight_decay: 0.0
), 'scheduler_query': <torch.optim.lr_scheduler.LambdaLR object at 0x7f5bd45888e0>, 'optimizer_passage': AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    initial_lr: 0.0001
    lr: 0.0
    weight_decay: 0.0
), 'scheduler_passage': <torch.optim.lr_scheduler.LambdaLR object at 0x7f5bd48716

Eval: 100%|██████████| 31/31 [00:01<00:00, 21.35it/s]


Eval loss: 3.440
Momento: [2023-Apr-18 00:18:12] Métricas iniciais em validação: {'valid/loss': 3.44012} Serão treinadas 1600 amostras


Epochs:   2%|▏         | 1/50 [02:29<2:02:14, 149.69s/it]

Eval loss: 0.221
Época: 1 Amostras:9984 de um total de 1600 (624.000%)
Momento: [2023-Apr-18 00:20:42] lr: 9.72759e-05 Treino loss: 1.0875 Validação loss: 0.2209  nova best epoca 0.22086751461029053


...

Epochs:  48%|████▊     | 24/50 [1:01:01<1:06:06, 152.55s/it]


Eval loss: 0.071
Parando por critério de early_stop no step 25 sendo best_epoca 5 e ealy_stop 20
Tempo gasto total 3661.29248, steps: 25, tempo por step 146.45170
Final: Step: 25 Amostras:249600  15600.000%  Momento: [2023-Apr-18 01:19:13] lr:0.00000e+00 Train loss: 0.0201  Validação loss: 0.0711 
Modelo com melhor resultado em validação (step 5) salvo após treino em /home/borela/fontes/deep_learning_em_buscas_unicamp/local/dpr_best_model_inicio_treino_2023-Apr-18 00:18:10


Eval: 100%|██████████| 31/31 [00:01<00:00, 21.28it/s]

Eval loss: 0.071
 Resultado com dados de teste para modelo treinado: {'valid/loss': 0.07109065}
0 {'loss_validacao': 0.07109065, 'loss_treino': 0.020110017, 'best_validation_loss': 0.07109065, 'best_epoca': 5}


[(0,
  {'loss_validacao': 0.07109065,
   'loss_treino': 0.020110017,
   'best_validation_loss': 0.07109065,
   'best_epoca': 5})]

#### batch size 32, lr menor, com rastro (zero_grad após chamada do modelo, antes da chamada de loss.backward)

In [174]:
# Se tiver que treinar os modelos, abre
models = {'query': AutoModel.from_pretrained(MODEL_NAME).to(hparam['device']),
'passage' : AutoModel.from_pretrained(MODEL_NAME).to(hparam['device'])}

In [175]:
gridparam = { 
               'learning_rate': [ 1e-5],
               'num_epochs':[50],
               'batch_size':[32],
             }             

In [176]:
treina_grid(hparam, gridparam, models, parm_se_gera_rastro = True, se_treina_poucos_dados=False)

Serão 1 experimentações


NUM: 1/1 : {'learning_rate': 1e-05, 'num_epochs': 50, 'batch_size': 32} 
Number of model parameters: 33360000


/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/tmp/ipykernel_2836714/1999415923.py:26: NeptuneDeprecationWarning: The object you're logging will be implicitly cast to a string. We'll end support of this behavior in `neptune-client==1.0.0`. To log the object as a string, use `str(object)` or `repr(object)` instead. For details, see https://docs.neptune.ai/setup/neptune-client_1-0_release_changes
  self.run_neptune['parameters'] = vparams


hparam: {'num_workers_dataloader': 0, 'device': device(type='cuda', index=0), 'batch_size': 32, 'num_epochs': 50, 'learning_rate': 1e-05, 'drop_last': True, 'train_size': 10000, 'valid_size': 1000, 'max_examples': 1600, 'eval_every_epocas': 1, 'early_stop': 30, 'criterion': <function compute_loss at 0x7f5bd4e9d700>, 'num_warmup_steps': 160, 'num_params': 33360000, 'optimizer_query': AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    initial_lr: 1e-05
    lr: 0.0
    weight_decay: 0.0
), 'scheduler_query': <torch.optim.lr_scheduler.LambdaLR object at 0x7f5bd4b21e50>, 'optimizer_passage': AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    initial_lr: 1e-05
    lr: 0.0
    weight_decay: 0.0
), 'scheduler_passage': <torch.optim.lr_scheduler.LambdaLR object at 0x7f5bd4b21e20>}


...

Eval loss: 0.090
Parando por critério de early_stop no step 35 sendo best_epoca 5 e ealy_stop 30
Tempo gasto total 5163.43968, steps: 35, tempo por step 147.52685
Final: Step: 35 Amostras:349440  21840.000%  Momento: [2023-Apr-18 10:24:10] lr:0.00000e+00 Train loss: 0.1051  Validação loss: 0.0901 
Modelo com melhor resultado em validação (step 5) salvo após treino em /home/borela/fontes/deep_learning_em_buscas_unicamp/local/dpr_best_model_inicio_treino_2023-Apr-18 08:58:05


Eval: 100%|██████████| 31/31 [00:01<00:00, 21.19it/s]

Eval loss: 0.090
 Resultado com dados de teste para modelo treinado: {'valid/loss': 0.09011188}
Shutting down background jobs, please wait a moment...


Done!
Waiting for the remaining 53 operations to synchronize with Neptune. Do not kill this process.
All 53 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/marcusborela/IA386DD/e/IAD-74
0 {'loss_validacao': 0.09011188, 'loss_treino': 0.10510916, 'best_validation_loss': 0.09011188, 'best_epoca': 5}


[(0,
  {'loss_validacao': 0.09011188,
   'loss_treino': 0.10510916,
   'best_validation_loss': 0.09011188,
   'best_epoca': 5})]

#### batch size 32, outro otimizador, zero_grad após step

In [116]:
# Se tiver que treinar os modelos, abre
models = {'query': AutoModel.from_pretrained(MODEL_NAME).to(hparam['device']),
'passage' : AutoModel.from_pretrained(MODEL_NAME).to(hparam['device'])}

In [122]:
torch.cuda.empty_cache()

In [123]:
gridparam = { 
               'learning_rate': [ 1e-5],
               'num_epochs':[50],
               'batch_size':[32],
             }             

In [124]:
treina_grid(hparam, gridparam, models, parm_se_gera_rastro = True, se_treina_poucos_dados=False)

Serão 1 experimentações de parâmetros


NUM: 1/1 : {'learning_rate': 1e-05, 'num_epochs': 50, 'batch_size': 32} 
Number of model parameters: 33360000


/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/tmp/ipykernel_19626/1999415923.py:12: NeptuneDeprecationWarning: `init` is deprecated, use `init_run` instead. We'll end support of it in `neptune-client==1.0.0`. For details, see https://docs.neptune.ai/setup/neptune-client_1-0_release_changes
  self.run_neptune = neptune.init(project=self.__class__.neptune_project, api_token=self.__class__.neptune_api_token, capture_hardware_metrics=True)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/tmp/ipykernel_19626/1999415923.py:26: NeptuneDeprecationWarning: The object you're logging will be implicitly cast to a string. We'll end support of this behavior in `neptune-client==1.0.0`. To log the object as a string, use `str(object)` or `repr(object)` instead. For details, see https://docs.neptune.ai/setup/neptune-client_1-0_release_changes
  self.run_neptune['parameters'] = vparams


hparam: {'num_workers_dataloader': 0, 'device': device(type='cuda', index=0), 'batch_size': 32, 'num_epochs': 50, 'learning_rate': 1e-05, 'drop_last': True, 'train_size': 10000, 'valid_size': 1000, 'steps_train_per_epoch': 312, 'max_examples': 500000, 'eval_every_epocas': 1, 'max_steps': 15600, 'early_stop': 20, 'criterion': <function compute_loss at 0x7f88ec367550>, 'num_warmup_steps': 1560, 'num_params': 33360000, 'optimizer_query': AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    initial_lr: 1e-05
    lr: 0.0
    weight_decay: 0.0
), 'scheduler_query': <torch.optim.lr_scheduler.LambdaLR object at 0x7f88ec385d60>, 'optimizer_passage': AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    initial_lr: 1e-05
    lr: 0.0
    weight_decay: 0.0
), 'scheduler_passage': <torch.optim.lr_scheduler.LambdaLR object at 0x7f88ec385d90>}


...

Eval loss: 0.087
Época: 50 Amostras:499200 de um total de 500000 (99.840%)
Momento: [2023-Apr-18 20:54:18] lr: 1.00000e-12 Treino loss: 0.0067 Validação loss: 0.0867 
Tempo gasto total 8217.08914, steps: 50, tempo por step 164.34178
Final: Step: 50 Amostras:499200  99.840%  Momento: [2023-Apr-18 20:54:18] lr:1.00000e-12 Train loss: 0.0067  Validação loss: 0.0867 
Modelo com melhor resultado em validação (step 34) salvo após treino em /home/borela/fontes/deep_learning_em_buscas_unicamp/local/dpr_best_model_inicio_treino_2023-Apr-18 18:37:19


Eval: 100%|██████████| 31/31 [00:01<00:00, 21.64it/s]

Eval loss: 0.087
 Resultado com dados de teste para modelo treinado: {'valid/loss': 0.08673091}
Shutting down background jobs, please wait a moment...


Done!
Waiting for the remaining 53 operations to synchronize with Neptune. Do not kill this process.
All 53 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/marcusborela/IA386DD/e/IAD-81
0 {'loss_validacao': 0.08673091, 'loss_treino': 0.0066537294, 'best_validation_loss': 0.079409346, 'best_epoca': 34}


[(0,
  {'loss_validacao': 0.08673091,
   'loss_treino': 0.0066537294,
   'best_validation_loss': 0.079409346,
   'best_epoca': 34})]

In [128]:
PATH_MODELO_FINAL = "/home/borela/fontes/deep_learning_em_buscas_unicamp/local/dpr/dpr_best_model_final"

In [131]:
for model in models:
    models[model].save_pretrained(f"{PATH_MODELO_FINAL}_{model}")

In [133]:
# Se tiver que treinar os modelos, abre
models_final = {'query': AutoModel.from_pretrained(f"{PATH_MODELO_FINAL}_query").to(hparam['device']),
'passage' : AutoModel.from_pretrained(f"{PATH_MODELO_FINAL}_passage").to(hparam['device'])}
print(models_final['query'].config.max_position_embeddings, models_final['passage'].config.vocab_size)
# 512 e 30522
#models['query'].config.__dict__


512 30522


In [134]:
valid_loaders = {
     'passage': DataLoader(valid_dataset, batch_size=hparam['batch_size'], shuffle=False, 
                           drop_last=True, num_workers=hparam['num_workers_dataloader'],
                           collate_fn=DPRCollator('passage', tokenizer)),
     'query': DataLoader(valid_dataset, batch_size=hparam['batch_size'], shuffle=False, 
                           drop_last=True, num_workers=hparam['num_workers_dataloader'],
                           collate_fn=DPRCollator('query', tokenizer))}

In [135]:
validation_loss = validation_step(models_final, valid_loaders)

Eval: 100%|██████████| 31/31 [00:01<00:00, 21.84it/s]

Eval loss: 0.087
